In [1]:
# @title Install libraries
!pip install sentence_transformers transformers accelerate bitsandbytes langchain einops
!pip install chromadb faiss-gpu pacmap

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.0/974.0 kB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.7/314.7 kB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.2/125.2 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.7/142.7 kB 2.1 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)

In [2]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# @title Load data

from io import BytesIO

import requests
import pandas as pd

gg_id = '814552626'
file_id = '11Xrvp1vdB0UJ34CIM_A-hKAnCJp_0BQA0HF6GGTerRc'
file_path = f'https://docs.google.com/spreadsheets/d/{file_id}/export?format=csv&gid={gg_id}'

df = pd.read_csv(file_path)
df = df.ffill()
df['Id'] = df.reset_index(drop=False)['index'].astype(str).str.zfill(5)
df

,Object,Detail,Interpretation,Id
0,Ảnh,Đốt ảnh,Tình duyên sắp thành,00000
1,Ảnh,Tặng ảnh,Điềm chia ly / sắp đi xa,00001
2,Ảnh,Được tặng ảnh,Điềm chia ly / sắp đi xa,00002
3,Ảnh,Ảnh người yêu,Có tình địch,00003
4,Ảnh,Xé ảnh người khác,Sắp hội ngộ người đó,00004
...,...,...,...,...
2303,Xưởng,Trông nom xưởng thở,Tài lộc hanh thông,02303
2304,Xưởng,Đi ra khỏi xưởng,Bạn sẽ được quý nhân cất nhắc,02304
2305,Xưởng,Đi ra khỏi xưởng rồi quay lại,Bạn đang được chủ tin cậy,02305
2306,Yêu,Yêu tha thiết 1 người mà người ấy luôn ruồng rẫy,Hôn nhân sắp thành,02306


In [4]:
doc_ids = df['Id'].tolist()
documents = [d.lower() for d in df['Detail'].tolist()]
metadatas = df.drop(columns=['Id','Detail']).to_dict(orient='records')
metadatas[:5]

[{'Object': 'Ảnh', 'Interpretation': 'Tình duyên sắp thành'},
 {'Object': 'Ảnh', 'Interpretation': 'Điềm chia ly / sắp đi xa'},
 {'Object': 'Ảnh', 'Interpretation': 'Điềm chia ly / sắp đi xa'},
 {'Object': 'Ảnh', 'Interpretation': 'Có tình địch'},
 {'Object': 'Ảnh', 'Interpretation': 'Sắp hội ngộ người đó'}]

In [5]:
# @title Load embedding model
from chromadb.utils.embedding_functions import SentenceTransformerEmbeddingFunction as EmbeddingFunction

model_name = 'bkai-foundation-models/vietnamese-bi-encoder'
embed_func = EmbeddingFunction(model_name=model_name)

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/6.46k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/895k [00:00<?, ?B/s]

bpe.codes:   0%|          | 0.00/1.14M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/22.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/167 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/270 [00:00<?, ?B/s]

In [6]:
# @title Build database
# Document: https://docs.trychroma.com/getting-started
%%time

import chromadb

client = chromadb.Client()
collection = client.get_or_create_collection(
    name="dreamerpretation",
    embedding_function=embed_func
)

collection.add(
          ids=doc_ids,
    documents=documents,
    metadatas=metadatas
)

collection.dict()

CPU times: user 57.4 s, sys: 167 ms, total: 57.6 s
Wall time: 58.4 s


{'name': 'dreamerpretation',
 'id': UUID('bb7c4ae9-9b21-4eb1-aeb6-0e411bda30b0'),
 'metadata': None,
 'tenant': 'default_tenant',
 'database': 'default_database'}

In [7]:
# @title Top-k Similarity Search
%%time

pd.options.display.float_format = '{:,.3f}'.format

queries = ["con mèo"]
Q = len(queries)
N = 7
results = collection.query(query_texts=queries, n_results=N)

q_idx = 0
results_processed = dict()
for k, v in results.items():
    if v is None:
        continue
    if k == 'metadatas':
        for d_ in v[q_idx]:
            for k_, v_ in d_.items():
                if k_ not in results_processed.keys():
                    results_processed[k_] = [v_]
                else:
                    results_processed[k_] += [v_]
    else:
        results_processed[k] = v[q_idx]

results_df = pd.DataFrame.from_records(results_processed)
results_df

CPU times: user 69.3 ms, sys: 5 µs, total: 69.3 ms
Wall time: 70.6 ms


,Interpretation,Object,distances,documents,ids
0,Cẩn thận những người bạn tin tưởng sẽ bội bạc ...,Mèo,0.000,con mèo,01355
1,Có sự do dự trong công việc,Cọp,38.540,con cọp,00506
2,Có nhiều triển vọng kiếm được tiền,Mèo,39.442,nuôi mèo,01361
3,Sắp có cãi vã với đàn ông. Nếu là phụ nữ có ch...,Chồn cáo,40.658,cáo,00467
4,Bạn sẽ có được người bạn tốt,Voi,40.710,voi,02249
5,Sắp gặp dịp may,Mèo,41.232,đuổi mèo đi,01357
6,"Bạn sẽ vượt qua được lời dèm pha, nói xấu",Mèo,41.778,giết mèo,01356


In [8]:
# @title Load language model
import torch
from transformers import pipeline
from transformers import AutoConfig, BitsAndBytesConfig
from transformers import AutoTokenizer, AutoModelForCausalLM
from sentence_transformers import SentenceTransformer

# bnb_config = BitsAndBytesConfig(
#     load_in_4bit=True,
#     bnb_4bit_use_double_quant=True,
#     bnb_4bit_quant_type="nf4",
#     bnb_4bit_compute_dtype=torch.bfloat16,
# )

# model_name = "vinai/PhoGPT-7B5-Instruct"
model_name = "vinai/PhoGPT-4B-Chat"
config = AutoConfig.from_pretrained(model_name, trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(model_name, trust_remote_code=True, config=config, load_in_8bit=True)

# reader = pipeline(
#     model=model,
#     tokenizer=tokenizer,
#     task="text-generation",
#     trust_remote_code=True,
#     do_sample=True,
#     temperature=0.169,
#     top_k=50,
#     top_p=0.11,
#     repetition_penalty=1.1,
#     return_full_text=True,
#     max_new_tokens=256,
# )

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/1.19k [00:00<?, ?B/s]

configuration_mpt.py:   0%|          | 0.00/16.4k [00:00<?, ?B/s]

fc.py:   0%|          | 0.00/167 [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/vinai/PhoGPT-4B-Chat:
- fc.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


...

A new version of the following files was downloaded from https://huggingface.co/vinai/PhoGPT-4B-Chat:
- adapt_tokenizer.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


meta_init_context.py:   0%|          | 0.00/3.96k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/vinai/PhoGPT-4B-Chat:
- meta_init_context.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/vinai/PhoGPT-4B-Chat:
- modeling_mpt.py
- hf_prefixlm_converter.py
- custom_embedding.py
- param_init_fns.py
- adapt_tokenizer.py
- meta_init_context.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
`low_cpu_mem_usage` was None, now set to True since model is quantized.


pytorch_model.bin:   0%|          | 0.00/7.38G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/91.0 [00:00<?, ?B/s]

In [9]:
# @title Context Combination
# Reference: https://github.com/VinAIResearch/PhoGPT
messages = []

# Combine ranked results as context
for symbol, meaning in results_df[['documents', 'Interpretation']].values:
    messages.append({"role": "user", "content": f"Ý nghĩa khi mơ thấy {symbol}",})
    messages.append({"role": "assistant", "content": meaning,})

messages.append({"role": "user", "content": f"Ý nghĩa khi mơ thấy {queries[0]}",})

# Using apply_chat_template
prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
print(prompt)


### Câu hỏi: Ý nghĩa khi mơ thấy con mèo
### Trả lời: Cẩn thận những người bạn tin tưởng sẽ bội bạc và lừa dối</s>
### Câu hỏi: Ý nghĩa khi mơ thấy con cọp
### Trả lời: Có sự do dự trong công việc</s>
### Câu hỏi: Ý nghĩa khi mơ thấy nuôi mèo
### Trả lời: Có nhiều triển vọng kiếm được tiền</s>
### Câu hỏi: Ý nghĩa khi mơ thấy cáo
### Trả lời: Sắp có cãi vã với đàn ông. Nếu là phụ nữ có chồng thì nên nhường nhịn. Nếu đang có người yêu thì nên giữ ý, coi chừng lời nói vô tình trở thành nguyên nhân gây gổ, thậm chí dẫn đến đổ vỡ mối quan hệ.</s>
### Câu hỏi: Ý nghĩa khi mơ thấy voi
### Trả lời: Bạn sẽ có được người bạn tốt</s>
### Câu hỏi: Ý nghĩa khi mơ thấy đuổi mèo đi
### Trả lời: Sắp gặp dịp may</s>
### Câu hỏi: Ý nghĩa khi mơ thấy giết mèo
### Trả lời: Bạn sẽ vượt qua được lời dèm pha, nói xấu</s>
### Câu hỏi: Ý nghĩa khi mơ thấy con mèo
### Trả lời:


In [11]:
# @title Answer Generation
# Reference: https://community.openai.com/t/cheat-sheet-mastering-temperature-and-top-p-in-chatgpt-api/172683

input_ids = tokenizer(prompt, return_tensors="pt")

outputs = model.generate(
                inputs = input_ids["input_ids"].to("cuda"),
        attention_mask = input_ids["attention_mask"].to("cuda"),
             do_sample = True,
           temperature = 0.169,
    repetition_penalty = 1.1,
                 top_k = 50,
                 top_p = 0.11,
        max_new_tokens = 128,
          eos_token_id = tokenizer.eos_token_id,
          pad_token_id = tokenizer.pad_token_id,
)

response = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
# response = response.split("### Trả lời:")[-1]
print(response)

### Câu hỏi: Ý nghĩa khi mơ thấy con mèo
### Trả lời: Cẩn thận những người bạn tin tưởng sẽ bội bạc và lừa dối
### Câu hỏi: Ý nghĩa khi mơ thấy con cọp
### Trả lời: Có sự do dự trong công việc
### Câu hỏi: Ý nghĩa khi mơ thấy nuôi mèo
### Trả lời: Có nhiều triển vọng kiếm được tiền
### Câu hỏi: Ý nghĩa khi mơ thấy cáo
### Trả lời: Sắp có cãi vã với đàn ông. Nếu là phụ nữ có chồng thì nên nhường nhịn. Nếu đang có người yêu thì nên giữ ý, coi chừng lời nói vô tình trở thành nguyên nhân gây gổ, thậm chí dẫn đến đổ vỡ mối quan hệ.
### Câu hỏi: Ý nghĩa khi mơ thấy voi
### Trả lời: Bạn sẽ có được người bạn tốt
### Câu hỏi: Ý nghĩa khi mơ thấy đuổi mèo đi
### Trả lời: Sắp gặp dịp may
### Câu hỏi: Ý nghĩa khi mơ thấy giết mèo
### Trả lời: Bạn sẽ vượt qua được lời dèm pha, nói xấu
### Câu hỏi: Ý nghĩa khi mơ thấy con mèo
### Trả lời: Bạn sắp bị phản bội bởi một ai đó thân thiết.

Bạn cũng dễ bị tổn thương trước các cuộc tấn công vật lý hoặc lời nói từ phía nam giới. 

Bạn cần cẩn trọng hơn về s